In [1]:
import os
import toml
import pandas as pd
import numpy as np
import plotly.express as px
import validation_data_input
import psrc_theme


import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook_connected" # to show plotly figures in quarto HTML file
pio.templates.default = "simple_white+psrc_color" # set plotly template

# run get_data.ipynb to load validation data
%store -r validation_data

config = toml.load(os.path.join(os.getcwd(), 'validation_configuration.toml'))

In [2]:
tour_data = validation_data.tours.copy()
atwork_subtour = tour_data.loc[tour_data['tour_category']=="atwork"].copy()

In [3]:
# aggregate transit modes
transit_modes = ['WALK_LOC','WALK_COM','WALK_FRY','WALK_LR','DRIVE_TRN']
tour_mode_ordered = ["DRIVEALONEFREE", "SHARED2FREE", "SHARED3FREE", "BIKE","WALK","ALL_TRANSIT","SCH_BUS","TNC","Other"]
atwork_subtour['tour_mode_transit_agg'] = atwork_subtour['tour_mode'].apply(lambda x: "ALL_TRANSIT" if x in transit_modes else x)

df_plot = atwork_subtour.groupby(['source','tour_mode_transit_agg'])['tour_weight'].sum().reset_index()
df_plot['percentage'] = df_plot.groupby(['source'], group_keys=False)['tour_weight']. \
    apply(lambda x: x / float(x.sum()))

fig = px.bar(df_plot, x="tour_mode_transit_agg", y="percentage", color="source",barmode="group",
             category_orders={"tour_mode_transit_agg": tour_mode_ordered},
             title="atwork subtour mode choice")
fig.for_each_annotation(lambda a: a.update(text = a.text.split("=")[-1]))
fig.update_layout(height=400, width=700, yaxis=dict(tickformat=".1%"))
fig.show()

In [4]:
df_plot = atwork_subtour.groupby(['source','tour_type','tour_mode_transit_agg'])['tour_weight'].sum().reset_index()
df_plot['percentage'] = df_plot.groupby(['source','tour_type'], group_keys=False)['tour_weight']. \
    apply(lambda x: x / float(x.sum()))

fig = px.bar(df_plot,
             x="tour_mode_transit_agg", y="percentage", color="source",barmode="group",
             facet_row='tour_type',
             category_orders={"tour_mode_transit_agg": tour_mode_ordered},
             title="atwork subtour mode choice by tour type")
fig.for_each_annotation(lambda a: a.update(text = a.text.split("=")[-1]))
fig.for_each_yaxis(lambda a: a.update(tickformat = ".1%"))
fig.update_layout(height=700, width=700)
fig.show()